<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/05-텍스트_생성.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  목차
* [Chapter 0 개요](#chapter0)
* [Chapter 1 일관성 있는 텍스트 생성의 어려움](#chapter1)
* [Chapter 2 그리디 서치 디코딩](#chapter2)

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. 트랜스포머 기반 언어 모델은 사람이 작성한 텍스트와 거의 구분되지 않는 텍스트를 생성한다.

2. 아래의 그림은 언어모델이 사전 훈련하는 동안 덧셈, 단어 철자 배열, 번역 같은 문맥 기반으로 다음 토큰을 예측하는 작업 시퀸스에 어떻게 노출되는지 보연준다.

   ![언어모델 훈련](image/05_00_language_modeling_tasks.png) 

## Chapter 1 일관성 있는 텍스트 생성의 어려움 <a class="anchor" id="chapter1"></a>
1. 모델의 확률 출력을 텍스트로 변환하려면 디코딩 방법이 필요한데, 텍스트 생성만 따르는 특수한 어려움이 있다.
    - 디코딩은 반복적으로 수행되므로 입력이 모델의 정방향 패스를 한 번 통과할 때보다 많은 계산이 필요하다.
    - 생성된 텍스트의 품직화 다양성은 디코딩 방법과 이에 관련되 하이퍼파리미터에 따라 달라진다.

2. GPT-2 코델은 문맥 시퀸스 x=x₁, x₂, ..., xₙ이 주어졌을 때 다음 토큰 xₙ₊₁의 확률 분포 P(xₙ₊₁|x)를 예측한다. 
    - 충분한 훈련 데이터를 획득하기란 어렵기 때문에, 일반적으호 확률 연쇄 법칙을 사용해 조건부 확률의 곱으로 전체 시퀸스의 확률을 계산한다.
    - P(x)=∏(i=1 to n) P(xᵢ|x₁, x₂, ..., xᵢ₋₁)

3. 조건부 확률로 자기회귀 언어 모델링은 문장의 이전 단어가 주어지면 다음 단어를 예측한다는 직관을 얻을 수 있다.
    - 이런 사전 훈련 목표는 과거와 미래의 문맥을 모두 사용해 마스킹된 토큰을 예측하는 BERT와 대조적이다.

4. 다음 토큰 예측 작업이 임의의 길이를 가진 텍스트 시퀸스를 생성할 대 어떻게 적용할지 예상해보자
    - "Transfomers are the"같은 프롬프트로 시작하면 모델은 다음  토큰을 예측한다.
    - 다음 토큰이 결저오디면 이를 프롬프트에 추가해 새로운 입력 시퀸스를 만들고 또 다른 토큰을 생성한다.
    - 이 과정을 원하는 길이나 종료 토큰을 만나기 전까지 텍스트 생성을 반복한다.
    - 출력 시퀸스가 입력 프롬프트에 따라 결정되므로 이런 종류의 텍스트 생성을 종종 조건부 텍스트 생성이라고 한다.
    
        ![텍스트 생성](image/05_01_text_generation.png)

5. 이 과정의 핵심은 각 타임스텝에서 어떤 토큰을 선택할지 결정하는 디코딩 방법에 있다.
    - 언어 모델의 해드는 각 스텝에서 어휘사전에 있는 토큰마다 로짓  zᵢ를 출력한다.
    - 그런 다음 소프트맥스 함수를 적용해 다음 토큰 wᵢ의 확률 분포를 계산한다.
        - P(yⱼ=wᵢ|y<ⱼ, x) = softmax(zⱼᵢ)
    - 대부분의 코딩 방법으 다음과 같은 ŷ을 선택해 전체적으로 확률이 가정 높은 시퀸스를 찾는다.
        - ŷ = argmax P(y|x) = argmax ∏(j=1 to m) P(yⱼ|y<ⱼ, x)
    - 직접 ŷ를 찾으려면 언어 모델로 가증한 모든 시퀸스를 평가해야 하므로 계산적으로 불가능하다.
    - 대신 근사 알고리즘을 사용해 효율적으로 높은 확률의 시퀸스를 찾는다.

## Chapter 2 그리디 서치 디코딩 <a class="anchor" id="chapter2"></a>
1. 그리디 서치 디코딩은 각 타임스텝에서 가장 높은 확률의 토큰을 선택하는 가장 단순한 디코딩 방법이다.
    - 이 방법은 각 타임스텝에서 지역적으로 최적의 선택을 하므로 전체적으로 최적의 시퀸스를 찾는다는 보장이 없다.
    - 그리디 서치 디코딩은 계산적으로 효율적이지만, 생성된 텍스트가 덜 다양하고 덜 창의적일 수 있다.
    - ŷⱼ = argmax P(yⱼ|y<ᵢ, x)


In [1]:
#15억개의 파라미터를 가진 GPT-2 모델 로드
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_namne = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_namne)
model = AutoModelForCausalLM.from_pretrained(model_namne).to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2026-01-15 09:46:13.873241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768437973.939407   15902 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768437973.959499   15902 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768437974.107692   15902 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768437974.107719   15902 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768437974.107721   15902 computation_placer.cc:177] computation placer alr

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

2. "Transformers are the"를 입력 프롬프트로 사용해 여덟 번의 타임스텝 동안 그리디 서치 디코딩을 수행해보자.
    - 각 타임스템에서 프롬프트의 마지막 토큰에 대한 로짓을 선택하고 소프트 맥스를 적용해 확률 분포를 계산한다.
    - 그런 다음 가장 높은 확률의 토큰을 선택해 프롬프트에 추가한다.
    - 이 과정을 여덟 번 반복하면 다음과 같은 토큰이 생성된다.
        - "Transformers are the most popular toy line in"

In [2]:
import pandas as pd

input_text = "Transformers are the"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
iterations = []
n_steps = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        outputs = model(input_ids=input_ids)
        # 첫 번째 배치의 마지막 토큰의 로짓을 선택해 소프트맥스를 적용한다.
        next_token_logits = outputs.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1,descending=True) 
        # 가장 높은 확률의 토큰을 저장한다.
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (f"{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)")
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # 예측한 다음 토큰을 입력에 추가한다.
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (8.53%),only (4.96%),best (4.65%),Transformers (4.37%),ultimate (2.16%)
1,Transformers are the most,popular (16.78%),powerful (5.37%),common (4.96%),famous (3.72%),successful (3.20%)
2,Transformers are the most popular,toy (10.63%),toys (7.23%),Transformers (6.60%),of (5.46%),and (3.76%)
3,Transformers are the most popular toy,line (34.38%),in (18.20%),of (11.71%),brand (6.10%),line (2.69%)
4,Transformers are the most popular toy line,in (46.28%),of (15.09%),", (4.94%)",on (4.40%),ever (2.72%)
5,Transformers are the most popular toy line in,the (65.99%),history (12.42%),America (6.91%),Japan (2.44%),North (1.40%)
6,Transformers are the most popular toy line in the,world (69.26%),United (4.55%),history (4.29%),US (4.23%),U (2.30%)
7,Transformers are the most popular toy line in ...,", (39.73%)",. (30.64%),and (9.87%),with (2.32%),today (1.74%)


In [5]:
# 트랜스포머스 내장 함수 gererate를 사용한 동일한 작업
input_text = "Transformers are the"
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"].to(device)
output_ids = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output_ids[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world,


In [7]:
# OpenAI의 유니콘 기사를 위한 그리디 서치 디코딩
max_length = 128
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able


3. OpenAI가 작성한 기사와 매우 다른다
    - 그리드 서치 알고리즘은 반복적인 출력 시퀸스를 생성하는 경향이 있다.
    - 확률이 높은 단어가 낮은 단어보다 먼저 등장하기 때문에 생성된 텍스트가 덜 다양하다.

4. 그리디 서치 디코딩은 다양성이 필요한 텍스트 생성 작업에는 거의 사용되지 않지만, 결정저이고 사실적으로 정확한 텍스트나 수식 생성이 필요한 작업에는 유용하다.